In [1]:
import datetime
import requests
import pymysql
import xml.etree.ElementTree as ET

In [2]:
day = datetime.date(2015,8,8)
end = datetime.date(2017,12,31)
delta = datetime.timedelta(1)
_url = 'http://www.cffex.com.cn/sj/hqsj/rtj/{0}/{1}/index.xml'
conn = pymysql.connect(
    host="219.224.169.45", 
    user="lizimeng", 
    password="codegeass", 
    db="market"
)
cursor = conn.cursor()
sql = "insert into contract_daily values(" + "%s,"*10 + "%s)"
while day<=end:
    # 确定url
    url = _url.format(day.strftime('%Y%m'), day.strftime('%d'))
    # 获取xml文件
    r = requests.get(url)
    # 判断是否真的获取了数据
    if r.url != url:
        day += delta
        continue
    # 解析xml & 写入文件
    root = ET.fromstring(r.text)
    data = []
    for child in root:
        vari = child[12].text.strip().lower()
        deli = child[0].text.strip()[-4:]
        close = float(child[5].text)
        oi = float(child[6].text)
        settle = float(child[9].text)
        volume = int(child[10].text)
        turnover = float(child[11].text)
        if child[2].text!=None:
            _open = float(child[2].text)
            high = float(child[3].text)
            low = float(child[4].text)
        else:
            _open = None
            high = None
            low = None
        cursor.execute(sql, (day,vari,deli,_open,high,low,close,settle,volume,oi,turnover))
    conn.commit()
    day += delta